In [ ]:
!pip install efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.0 MB/s eta 0:00:00


In [ ]:
!wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Train_mscoco.zip

--2023-05-09 22:01:59--  https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Train_mscoco.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.95.88, 52.217.97.70, 52.216.216.96, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.95.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7239401 (6.9M) [application/zip]
Saving to: ‘v2_Questions_Train_mscoco.zip’

v2_Questions_Train_ 100%[===================>]   6.90M  31.0MB/s    in 0.2s    

2023-05-09 22:02:00 (31.0 MB/s) - ‘v2_Questions_Train_mscoco.zip’ saved [7239401/7239401]



In [ ]:
!unzip "/content/v2_Questions_Train_mscoco.zip"

Archive:  /content/v2_Questions_Train_mscoco.zip
  inflating: v2_OpenEnded_mscoco_train2014_questions.json  


In [ ]:
!wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip

--2023-05-09 22:02:00--  https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.14.150, 52.217.230.40, 52.216.219.144, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.14.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21708861 (21M) [application/zip]
Saving to: ‘v2_Annotations_Train_mscoco.zip’

v2_Annotations_Trai 100%[===================>]  20.70M  34.3MB/s    in 0.6s    

2023-05-09 22:02:01 (34.3 MB/s) - ‘v2_Annotations_Train_mscoco.zip’ saved [21708861/21708861]



In [ ]:
!unzip "/content/v2_Annotations_Train_mscoco.zip"

Archive:  /content/v2_Annotations_Train_mscoco.zip
  inflating: v2_mscoco_train2014_annotations.json  


In [ ]:
import json
import tensorflow as tf
from efficientnet.tfkeras import EfficientNetB2, preprocess_input
from transformers import BertTokenizer, TFBertForQuestionAnswering
import pickle
import re
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.utils import load_img, img_to_array
from tqdm import tqdm

In [ ]:
# Read dictionary pkl file
with open('/content/drive/MyDrive/FInal Project NTI/train_image_feature.pkl', 'rb') as fp:
    train_imgs_features = pickle.load(fp)
    print('successful')

successful


In [ ]:
train_imgs_features

{9: array([0.3641575 , 0.39742967, 0.848959  , ..., 0.07714461, 0.5495399 ,
        0.5432545 ], dtype=float32),
 25: array([0.2270625 , 0.3399307 , 0.25919005, ..., 0.18876874, 0.25891095,
        0.1660124 ], dtype=float32),
 30: array([0.28736764, 0.65010697, 1.5415224 , ..., 0.23868492, 1.1545415 ,
        0.0908316 ], dtype=float32),
 34: array([1.5141456 , 0.08357344, 0.03438409, ..., 0.54353356, 0.3973285 ,
        0.02907126], dtype=float32),
 36: array([1.0456916 , 0.17438439, 0.49074018, ..., 0.7129947 , 0.77394235,
        0.05939685], dtype=float32),
 49: array([0.19489115, 0.14706653, 0.16575038, ..., 0.19204912, 0.29227567,
        0.5584221 ], dtype=float32),
 61: array([0.03941727, 0.1623201 , 0.5579196 , ..., 0.77507114, 0.39080268,
        0.05651416], dtype=float32),
 64: array([0.07590349, 0.17269596, 0.14271513, ..., 0.9711013 , 0.1546471 ,
        0.4162653 ], dtype=float32),
 71: array([0.5848455 , 1.094417  , 0.06460974, ..., 0.08701868, 0.21991473,
        0.01

In [ ]:
#train_imgs_features{7224}

In [ ]:
print(train_imgs_features.values)

<built-in method values of dict object at 0x7fc2eb29aa40>


In [ ]:
list(train_imgs_features.keys())

[9,
 25,
 30,
 34,
 36,
 49,
 61,
 64,
 71,
 72,
 77,
 78,
 81,
 86,
 89,
 92,
 94,
 109,
 110,
 113,
 127,
 138,
 142,
 144,
 149,
 151,
 154,
 165,
 194,
 201,
 247,
 250,
 260,
 263,
 307,
 308,
 309,
 312,
 315,
 321,
 322,
 326,
 332,
 349,
 368,
 370,
 382,
 384,
 389,
 394,
 404,
 419,
 431,
 436,
 438,
 443,
 446,
 450,
 471,
 490,
 491,
 508,
 510,
 514,
 529,
 531,
 532,
 540,
 542,
 560,
 562,
 572,
 575,
 581,
 584,
 595,
 597,
 605,
 612,
 620,
 625,
 629,
 634,
 643,
 650,
 656,
 659,
 670,
 671,
 673,
 681,
 684,
 690,
 706,
 714,
 716,
 722,
 723,
 731,
 735,
 753,
 754,
 762,
 781,
 790,
 795,
 797,
 801,
 813,
 821,
 825,
 828,
 839,
 853,
 882,
 897,
 901,
 902,
 908,
 909,
 913,
 925,
 927,
 934,
 941,
 943,
 955,
 960,
 965,
 977,
 982,
 984,
 996,
 1006,
 1011,
 1014,
 1025,
 1036,
 1053,
 1059,
 1072,
 1084,
 1085,
 1090,
 1098,
 1099,
 1102,
 1107,
 1108,
 1111,
 1122,
 1139,
 1144,
 1145,
 1155,
 1166,
 1168,
 1183,
 1200,
 1204,
 1213,
 1216,
 1224,
 1232,
 12

In [ ]:
#train_imgs_features.values()

In [ ]:
# Define the paths to your input files
question_file_path = "/content/v2_OpenEnded_mscoco_train2014_questions.json"
annotation_file_path = "/content/v2_mscoco_train2014_annotations.json"

# Load the question file and extract the question data
with open(question_file_path, "r") as f:
    question_data = json.load(f)["questions"]

# Load the annotation file and extract the ground truth answers
with open(annotation_file_path, "r") as f:
    annotation_data = json.load(f)["annotations"]

In [ ]:
# Group the question and answer data by img_id using a dictionary comprehension
qa_data = {a['image_id']: {'questions': [], 'answers': []} for a in annotation_data}

In [ ]:
for q in question_data:
    qa_data[q['image_id']]['questions'].append(q['question'])

In [ ]:
for a in annotation_data:
    qa_data[a['image_id']]['answers'].append(a['multiple_choice_answer'])

In [ ]:
# Generate the input data using a list comprehension
input_data = [{"image": train_imgs_features[key], "question": q, "answer": a} 
              for key in list(train_imgs_features.keys()) 
              for q, a in zip(qa_data[key]['questions'], qa_data[key]['answers'])]

In [ ]:
print(input_data[150])

{'image': array([0.44045335, 0.50492954, 0.14766406, ..., 0.22389303, 0.21844536,
       0.28869957], dtype=float32), 'question': 'Where was picture taken?', 'answer': 'park'}


In [ ]:
# Load the pre-trained models
efnet_model = EfficientNetB2(weights='imagenet')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertForQuestionAnswering.from_pretrained('bert-base-uncased')

37424016/37424016 [==============================] - 0s 0us/step


All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize and encode the question data using the appropriate tokenizer
encoded_data = []
for d in input_data:
    question_tokens = bert_tokenizer(d["question"], padding=True, truncation=True, return_tensors="tf")
    encoded_data.append({"image": d["image"], "question": question_tokens, "answer": d["answer"]})



In [ ]:
# Pass the preprocessed input data to the pre-trained models to get the predicted answers
bert_outputs = []
for d in encoded_data:
    #efnet_output = tf.keras.layers.Flatten()(d["image"])
    efnet_output = d["image"]
    print(efnet_output)
    bert_input = {"input_ids": d["question"]["input_ids"], "attention_mask": d["question"]["attention_mask"]}#, 'image': efnet_output }
    print(bert_input)
    bert_output = bert_model(bert_input)
    bert_outputs.append(tf.argmax(bert_output.start_logits, axis=-1).numpy())

Streaming output truncated to the last 5000 lines.
array([[  101,  2054,  2003,  2369,  1996, 12586,  2121,  1029,   102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
[0.36214322 0.11902103 0.22257867 ... 0.35679483 0.26675975 0.2538435 ]
{'input_ids': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=
array([[  101,  2003,  2045,  1037,  5404, 14757,  2006,  1996,  2795,
         1029,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
[0.36214322 0.11902103 0.22257867 ... 0.35679483 0.26675975 0.2538435 ]
{'input_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=
array([[  101,  2054,  3609,  2003,  1996, 18097,  1029,   102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
[0.12593982 0.40667284 0.49511993 ... 0.3784

In [ ]:
# # Pass the preprocessed input data to the pre-trained models to get the predicted answers
# bert_outputs = []
# for d in encoded_data:
#     #efnet_output = tf.keras.layers.Flatten()(d["image"])
#     efnet_output = d["image"]
#     #efnet_output = tf.reshape(efnet_output, (1, 1, 1, -1))  # Reshape to match the expected shape of the image input
#     bert_input = {"input_ids": d["question"]["input_ids"], "attention_mask": d["question"]["attention_mask"], "image": efnet_output}
#     bert_output = bert_model(bert_input)
#     bert_outputs.append(tf.argmax(bert_output.logits, axis=-1).numpy()[0])

In [ ]:
#evaluate model